<a href="https://colab.research.google.com/github/yashveersinghsohi/Dive_into_Deep_Learning_Practice/blob/chapter_3.3/chapter_3/ch_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Book Link](https://d2l.ai/chapter_linear-regression/synthetic-regression-data.html)

# Imports

In [2]:
# !pip install git+https://github.com/d2l-ai/d2l-en.git

In [3]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

# Dataset

In [4]:
class SyntheticRegressionData(d2l.DataModule): 
    """Synthetic data for linear regression."""
    def __init__(self, w, b, noise=0.01, num_train=1000, num_val=1000,
                 batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise

In [5]:
data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)

In [6]:
print('features:', data.X[0],'\nlabel:', data.y[0])

features: tensor([-0.3289,  0.2201]) 
label: tensor([2.7904])


# Data loader from scratch

In [7]:
@d2l.add_to_class(SyntheticRegressionData)
def get_dataloader(self, train):
    if train:
        indices = list(range(0, self.num_train))
        # The examples are read in random order
        random.shuffle(indices)
    else:
        indices = list(range(self.num_train, self.num_train+self.num_val))
    for i in range(0, len(indices), self.batch_size):
        batch_indices = torch.tensor(indices[i: i+self.batch_size])
        yield self.X[batch_indices], self.y[batch_indices]

In [8]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


# Concise Data Loader

In [9]:
@d2l.add_to_class(d2l.DataModule) 
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    dataset = torch.utils.data.TensorDataset(*tensors)
    return torch.utils.data.DataLoader(dataset, self.batch_size,
                                       shuffle=train)

@d2l.add_to_class(SyntheticRegressionData) 
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)

In [10]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


In [11]:
len(data.train_dataloader())

32